In [15]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [23]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 500 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 20
piter = 0
batch_size = 3 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
prior_scale_factor = 0.25 #Prior standard deviations set at 0.25 of prior means.
obs_error_scale_factor = 0.1

In [24]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [25]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [26]:
#Mean field VI code block.

#Define mean-field class: consumes parameter dictionary with values used as initial mean values.
class MeanField(nn.Module):
    def __init__(self, init_params, sdev_scale):
        super().__init__()

        #Use param dict to intialise the means for the mean-field approximations.
        means = []
        keys = []
        for key, value in init_params.items():
            keys += [key]
            means += [value]
        self.means = nn.Parameter(torch.Tensor(means))
        self.sds = nn.Parameter(self.means * sdev_scale)
        #Save keys for forward output.
        self.keys = keys

    def forward(self, n = 30):
        #Update posterior.
        q_dist = D.normal.Normal(self.means, LowerBound.apply(self.sds, 1e-7))
        #Sample theta ~ q(theta).
        samples = q_dist.rsample([n])
        # evaluate log prob of theta samples
        log_q_theta = torch.sum(q_dist.log_prob(samples), -1) # shape of n
        # return samples in same dictionary format
        dict_out = {} # define dictionary with n samples for each parameter
        for key, sample in zip(self.keys, torch.split(samples, 1, -1),):
            dict_out[f"{key}"] = sample.squeeze(1) # each sample is of shape [n]
        # returns samples in dictionary and tensor format
        return dict_out, samples, log_q_theta

In [27]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [28]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, PRIOR_SCALE_FACTOR, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE_FACTOR, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAM_PRIOR_MEANS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE_FACTOR) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE)
    prior_means_tensor = torch.Tensor(list(PARAM_PRIOR_MEANS_DICT.values()))
    priors = D.normal.Normal(prior_means_tensor, prior_means_tensor * PRIOR_SCALE_FACTOR)
    q_theta = MeanField(PARAM_PRIOR_MEANS_DICT, PRIOR_SCALE_FACTOR)
    pretrain_optimizer = optim.Adamax(net.parameters(), lr = L_R, eps = 1e-7)
    ELBO_optimizer = optim.Adam(list(net.parameters()) + list(q_theta.parameters()), lr = L_R)
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            theta_dict, theta, log_q_theta = q_theta(BATCH_SIZE)
            log_p_theta = priors.log_prob(theta).sum()
            C_0 = LowerBound.apply(ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), theta_dict), 1e-5) #Calculate deterministic initial conditions.
            print('C_0 =', C_0)
            #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Commenting out because analytical steady state init functions now output tensors with appropriate batch size if argument into MeanField forward function is BATCH_SIZE. #Assign initial conditions to C_PATH.
            C_PATH = torch.cat([C_0.unsqueeze(1), C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                pretrain_optimizer.zero_grad()
                #l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                #l1_norm.backward()
                l2_norm.backward()
                pretrain_optimizer.step()
            else:
                ELBO_optimizer.zero_grad()
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, theta_dict, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_p_theta + log_q_theta - log_lik.mean() - obs_model(C_PATH) + log_prob.mean() #From equation 14 of Ryder et al., 2019.
                neg_ELBO_mean = neg_ELBO.mean()
                print('neg_ELBO_mean = ', neg_ELBO_mean)
                best_loss_ELBO = neg_ELBO_mean if neg_ELBO_mean < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO_mean)
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                neg_ELBO_mean.backward()
                ELBO_optimizer.step()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            if iter % 100000 == 0 and iter > 0:
                ELBO_optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [29]:
train(devi, l_r, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/20 [00:00<?, ?it/s]

C_0 = tensor([[8.4661e+01, 7.2928e-02, 5.1392e-01],
        [5.1583e+01, 5.4876e-02, 8.5066e-01],
        [6.9496e+01, 6.6474e-02, 9.6074e-01]], grad_fn=<LowerBoundBackward>)
Moving average norm loss at 0 iterations is: 9000000388.15327. Best norm loss value is: 3881.53271484375.

C_PATH mean = tensor([[0.7504, 0.6638, 0.7868],
        [0.7452, 0.6691, 0.7844],
        [0.7488, 0.6655, 0.7934]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[8.4661e+01, 7.2928e-02, 5.1392e-01],
         [7.1664e-01, 8.1981e-01, 6.9775e-01],
         [6.1944e-01, 1.1945e+00, 1.2938e+00],
         ...,
         [9.6820e-01, 6.7592e-01, 5.8883e-01],
         [8.0403e-01, 7.9684e-01, 4.3321e-01],
         [1.0562e+00, 4.5107e-01, 9.7814e-02]],

        [[5.1583e+01, 5.4876e-02, 8.5066e-01],
         [7.1664e-01, 7.1837e-01, 1.1739e+00],
         [9.2789e-01, 5.3652e-01, 5.6271e-01],
         ...,
         [5.7009e-01, 7.2633e-01, 9.5856e-01],
         [6.6098e-01, 5.6412e-01, 3.9962e-01],
         [6.7344e-0


Train Diffusion:   5%|▌         | 1/20 [00:03<01:11,  3.75s/it]

C_0 = tensor([[5.9267e+01, 7.8779e-02, 4.5454e-01],
        [2.9836e+01, 4.9183e-02, 7.0845e-01],
        [4.2896e+01, 8.5888e-02, 1.4140e+00]], grad_fn=<LowerBoundBackward>)
neg_ELBO_mean =  tensor(385018.9688, grad_fn=<MeanBackward0>)



Train Diffusion:  10%|█         | 2/20 [00:07<01:05,  3.66s/it]

...


Train Diffusion:  90%|█████████ | 18/20 [00:59<00:07,  3.66s/it]

C_0 = tensor([[1.0000e-05, 1.0055e-01, 2.1084e+00],
        [1.0000e-05, 5.7214e-02, 1.0000e-05],
        [1.0000e-05, 3.0818e-02, 1.0000e-05]], grad_fn=<LowerBoundBackward>)
neg_ELBO_mean =  tensor(151814.2031, grad_fn=<MeanBackward0>)



Train Diffusion:  95%|█████████▌| 19/20 [01:03<00:03,  3.65s/it]

C_0 = tensor([[1.0000e-05, 2.6991e-02, 1.0000e-05],
        [1.0000e-05, 3.6857e-02, 1.0000e-05],
        [1.0000e-05, 4.2835e-02, 1.0000e-05]], grad_fn=<LowerBoundBackward>)
neg_ELBO_mean =  tensor(151627.5625, grad_fn=<MeanBackward0>)



Train Diffusion: 100%|██████████| 20/20 [01:06<00:00,  3.32s/it]


In [11]:
t_span_tensor.size()

torch.Size([1, 2501, 1])

In [15]:
prior_means_tensor = torch.Tensor(list(SCON_C_prior_means.values()))
priors = D.normal.Normal(prior_means_tensor, prior_means_tensor * prior_scale_factor)
q_theta = MeanField(SCON_C_prior_means, prior_scale_factor)
theta_dict, theta, log_q_theta = q_theta(batch_size)

In [16]:
theta_dict

{'u_M': tensor([0.0016, 0.0023], grad_fn=<SqueezeBackward1>),
 'a_SD': tensor([0.2647, 0.2314], grad_fn=<SqueezeBackward1>),
 'a_DS': tensor([0.2608, 0.3378], grad_fn=<SqueezeBackward1>),
 'a_M': tensor([0.4350, 0.1587], grad_fn=<SqueezeBackward1>),
 'a_MSC': tensor([0.3455, 0.3346], grad_fn=<SqueezeBackward1>),
 'k_S_ref': tensor([1.8679e-05, 2.9906e-05], grad_fn=<SqueezeBackward1>),
 'k_D_ref': tensor([0.0049, 0.0061], grad_fn=<SqueezeBackward1>),
 'k_M_ref': tensor([0.0002, 0.0002], grad_fn=<SqueezeBackward1>),
 'Ea_S': tensor([62.2977, 91.1898], grad_fn=<SqueezeBackward1>),
 'Ea_D': tensor([33.3193, 42.6860], grad_fn=<SqueezeBackward1>),
 'Ea_M': tensor([52.0719, 29.6192], grad_fn=<SqueezeBackward1>),
 'c_SOC': tensor([1.1225, 1.0827], grad_fn=<SqueezeBackward1>),
 'c_DOC': tensor([0.0041, 0.0146], grad_fn=<SqueezeBackward1>),
 'c_MBC': tensor([0.0881, 0.1043], grad_fn=<SqueezeBackward1>)}

In [17]:
obs_times, obs_means, obs_error = csv_to_obs_df('CON_synthetic_sol_df.csv', 3 + 1, t, obs_error_scale_factor)
obs_model = ObsModel(devi, obs_times, dt, obs_means[:-1, :], obs_error[:, :-1])
net = SDEFlow(devi, batch_size, obs_model, 3, t, dt, n).to(devi)

In [18]:
C_PATH, log_prob = net()
print(C_PATH)
C_PATH.size()

tensor([[[0.6963, 0.8716, 0.8377],
         [0.9976, 0.8534, 0.5846],
         [0.5680, 0.7184, 0.7918],
         ...,
         [0.7768, 0.6447, 0.4190],
         [1.0818, 0.6042, 0.3987],
         [0.7856, 0.5718, 0.3353]],

        [[0.6963, 0.9099, 0.5898],
         [1.0600, 0.7288, 0.5244],
         [0.7498, 1.1136, 0.5916],
         ...,
         [0.7668, 0.4979, 0.4380],
         [0.5906, 0.5252, 0.3187],
         [1.0386, 0.5509, 0.0981]]], grad_fn=<AddBackward0>)


torch.Size([2, 2500, 3])

In [20]:
C_0 = analytical_steady_state_init_CON(i_s_tensor[0, 0, 0].item(), i_d_tensor[0, 0, 0].item(), theta_dict)
print(C_0)
C_0.unsqueeze(1)

tensor([[5.8788e+01, 6.4810e-02, 4.3640e-01],
        [3.6589e+01, 4.3350e-02, 5.1894e-01]], grad_fn=<StackBackward>)


tensor([[[5.8788e+01, 6.4810e-02, 4.3640e-01]],

        [[3.6589e+01, 4.3350e-02, 5.1894e-01]]], grad_fn=<UnsqueezeBackward0>)

In [21]:
C_PATH = torch.cat([C_0.unsqueeze(1), C_PATH], 1)

In [22]:
SOC, DOC, MBC =  torch.chunk(C_PATH, 3, -1)
SOC.size()

torch.Size([2, 2501, 1])

In [25]:
current_temp = temp_gen(t_span_tensor, temp_ref)

In [26]:
theta_dict['k_S_ref'].size()

torch.Size([2])

In [30]:
k_S = arrhenius_temp_dep(theta_dict['k_S_ref'], current_temp, theta_dict['Ea_S'], temp_ref)
k_S = k_S.permute(2, 1, 0)
k_D = arrhenius_temp_dep(theta_dict['k_D_ref'], current_temp, theta_dict['Ea_D'], temp_ref)
k_D = k_D.permute(2, 1, 0)
k_M = arrhenius_temp_dep(theta_dict['k_M_ref'], current_temp, theta_dict['Ea_M'], temp_ref)
k_M = k_M.permute(2, 1, 0)

In [36]:
theta_dict_repeat = dict((k, v.repeat(1, 2501, 1).permute(2, 1, 0)) for k, v in theta_dict.items())
drift_SOC = i_s_tensor + theta_dict_repeat['a_DS'] * k_D * DOC + theta_dict_repeat['a_M'] * theta_dict_repeat['a_MSC'] * k_M * MBC - k_S * SOC
drift_DOC = i_d_tensor + theta_dict_repeat['a_SD'] * k_S * SOC + theta_dict_repeat['a_M'] * (1 - theta_dict_repeat['a_MSC']) * k_M * MBC - (theta_dict_repeat['u_M'] + k_D) * DOC
drift_MBC = theta_dict_repeat['u_M'] * DOC - k_M * MBC

In [37]:
drift_MBC.size()

torch.Size([2, 2501, 1])

In [73]:
theta_dict_u_M_test = theta_dict['u_M'].repeat(1, 2501, 1)
theta_dict_u_M_test = theta_dict_u_M_test.permute(2, 1, 0)
print(theta_dict_u_M_test.size())
test = theta_dict_u_M_test * DOC
print(test.size())

torch.Size([2, 2501, 1])
torch.Size([2, 2501, 1])


In [24]:
C_PATH.size()

torch.Size([2, 2501, 3])

In [38]:
drift = torch.empty_like(C_PATH, device = C_PATH.device) #Initiate tensor with same dims as C_PATH to assign drift.
drift[:, :, 0 : 1] = drift_SOC
drift[:, :, 1 : 2] = drift_DOC
drift[:, :, 2 : 3] = drift_MBC

In [60]:
test_m = torch.stack([theta_dict['c_SOC'], theta_dict['c_DOC'], theta_dict['c_MBC']], 1)
print(LowerBound.apply(test_m, 1e-6))
test_m_sqrt = torch.sqrt(test_m)
torch.diag_embed(test_m_sqrt)

tensor([[1.1225, 0.0041, 0.0881],
        [1.0827, 0.0146, 0.1043]], grad_fn=<LowerBoundBackward>)


tensor([[[1.0595, 0.0000, 0.0000],
         [0.0000, 0.0639, 0.0000],
         [0.0000, 0.0000, 0.2969]],

        [[1.0406, 0.0000, 0.0000],
         [0.0000, 0.1208, 0.0000],
         [0.0000, 0.0000, 0.3230]]], grad_fn=<CopySlices>)

In [64]:
diffusion_sqrt_single = torch.diag_embed(torch.sqrt(LowerBound.apply(torch.stack([theta_dict['c_SOC'], theta_dict['c_DOC'], theta_dict['c_MBC']], 1), 1e-6)))
diffusion_sqrt_single

tensor([[[1.0595, 0.0000, 0.0000],
         [0.0000, 0.0639, 0.0000],
         [0.0000, 0.0000, 0.2969]],

        [[1.0406, 0.0000, 0.0000],
         [0.0000, 0.1208, 0.0000],
         [0.0000, 0.0000, 0.3230]]], grad_fn=<CopySlices>)

In [80]:
diffusion_sqrt_single.unsqueeze(1).expand(-1, 2501, -1, -1)

tensor([[[[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]],

         [[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]],

         [[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]],

         ...,

         [[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]],

         [[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]],

         [[1.0595, 0.0000, 0.0000],
          [0.0000, 0.0639, 0.0000],
          [0.0000, 0.0000, 0.2969]]],


        [[[1.0406, 0.0000, 0.0000],
          [0.0000, 0.1208, 0.0000],
          [0.0000, 0.0000, 0.3230]],

         [[1.0406, 0.0000, 0.0000],
          [0.0000, 0.1208, 0.0000],
          [0.0000, 0.0000, 0.3230]],

         [[1.0406, 0.0000, 0.0000],
          [0.0000, 0.1208, 0.0000],
          [0.0000, 0.0000, 0.32